In [0]:
dbutils.fs.cp("file:/Workspace/Shared/sales data.csv", "dbfs:/FileStore/streaming/input/sales data.csv")
dbutils.fs.cp("file:/Workspace/Shared/customer d .json", "dbfs:/FileStore/streaming/input/customer data.json")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2244438965545007>:1
----> 1 dbutils.fs.cp("file:/Workspace/Shared/sales data.csv", "dbfs:/FileStore/streaming/input/sales data.csv")
      2 dbutils.fs.cp("file:/Workspace/Shared/customer d .json", "dbfs:/FileStore/streaming/input/customer data.json")

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o1158.cp.
: java.io.FileNotFoundException: File file:/Workspace/Shared/sales data.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)


In [0]:
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data2.csv")

In [0]:
df2 = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/customer_data.json")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#Initialize SparkSession
spark = SparkSession.builder \
.appName("StructuredStreamingExample") \
.getOrCreate()

#Define the schema for the CSV data
sales_schema = "OrderID INT, OrderDate STRING, CustomerID STRING, Product STRING, Quantity INT, Price DOUBLE"

#Read streaming data from CSV files
df1 = spark.readStream \
.format("CSV") \
.option("header", "true") \
.schema (sales_schema) \
.load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/")

#Define the schema for the JSON data
customer_schema = "CustomerID STRING, CustomerName STRING, Region STRING, SignupDate STRING"

#Read streaming data from JSON files
df2 = spark.readStream \
.format("json") \
.schema (customer_schema) \
.load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/")

#df2_customers_stream.print
df2.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2244438965545010>", line 29, in <module>
    df2.show()
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/dataframe.py", line 920, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/]

During handling of the above except

In [0]:
# Define the schema for the streaming data
from pyspark.sql.types import StructType,IntegerType, StructField, StringType, TimestampType, FloatType,DoubleType

schema = StructType([
    StructField("OrderID", IntegerType(), True),
    StructField("OrderDate", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Product", FloatType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", DoubleType(), True)
])

# Read the CSV file as a streaming DataFrame
input_path = "dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data2.csv"

# Read the file as if it's a stream
df1_stream = spark.readStream \
    .option("header", "true") \
    .schema(schema) \
    .csv(input_path)

# Display the streaming DataFrame (for debugging purposes)
df1_stream.printSchema()


root
 |-- OrderID: integer (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Product: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)



In [0]:
# Define the schema for the streaming data
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("CustomerName ", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("SignupDate", StringType(), True)
    
])

# Read the CSV file as a streaming DataFrame
input_path = "dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/customer_data.json"

# Read the file as if it's a stream
df2_stream = spark.readStream \
    .option("header", "true") \
    .schema(schema) \
    .csv(input_path)

# Display the streaming DataFrame (for debugging purposes)
df2_stream.printSchema()


root
 |-- CustomerID: string (nullable = true)
 |-- CustomerName : string (nullable = true)
 |-- Region: string (nullable = true)
 |-- SignupDate: string (nullable = true)



In [0]:
from pyspark.sql.functions import current_date, datediff, to_timestamp

#Transform the sales data: Add a new column for total amount
df_sales_transformed = df1_stream.select(
col ("OrderID"),
to_timestamp (col ("OrderDate"), "yyyy-MM-dd HH:mm:ss").alias("OrderDate"), #Convert OrderDate to TIMESTAMP
col ("Product"),
col ("Quantity"),
col ("Price"),
(col ("Quantity") * col("Price")).alias ("TotalAmount")
)

print("Applied transformations on sales data...")

#Add watermark to handle late data and perform an aggregation
df_sales_aggregated = df_sales_transformed \
    .withWatermark("OrderDate", "1 day") \
    .groupBy("Product") \
    .agg({"TotalAmount": "sum"})

print("Aggregated sales data by product...")

#Transform the customer data: Add a new column for the number of years since signup 
df_customers_transformed = df2_stream.withColumn (
    "YearsSinceSignup",
    datediff (current_date(), to_timestamp (col ("SignupDate"), "yyyy-MM-dd")).cast("int") / 365
)

print("Applied transformations on customer data...")

Applied transformations on sales data...
Aggregated sales data by product...
Applied transformations on customer data...


In [0]:
# Write the aggregated sales data to a console sink for debugging
sales_query= df_sales_aggregated.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

print("Started streaming query to write aggregated sales data to console...")

# Write the transformed customer data to a console sink for debugging
customers_query= df_customers_transformed.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()
    
print("Started streaming query to write transformed customer data to console...")

Started streaming query to write aggregated sales data to console...
Started streaming query to write transformed customer data to console...


In [0]:
df3 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data3.csv")

In [0]:
import pandas as pd

#Create sample sales data
data = {
"OrderID": [1, 2, 3, 4], 
"OrderDate": ["2024-01-01 10:00:00", "2024-01-02 11:00:00", "2024-01-03 12:00:00", "2024-01-04 13:00:00"], 
"CustomerID": ["C001", "C002", "C003", "C004"], 
"Product": ["ProductA", "ProductB", "ProductC", "ProductD"], 
"Quantity": [10, 20, 15, 5],
"Price": [100.0, 200.0, 150.0, 50.0]
}

#Convert to DataFrame
df =  pd.DataFrame(data)

#Save to CSV
csv_path="/dbfs/Shared/data/sales_data.csv" 
df.to_csv(csv_path, index=False)

print(f"Sample data saved to {csv_path}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2244438965545017>", line 18, in <module>
    df.to_csv(csv_path, index=False)
  File "/databricks/python/lib/python3.9/site-packages/pandas/core/generic.py", line 3551, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/databricks/python/lib/python3.9/site-packages/pandas/io/formats/format.py", line 1180, in to_csv
    csv_formatter.save()
  File "/databricks/python/lib/python3.9/site-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "/databricks/python/lib/python3.9/site-packages/pandas/io/common.py", line 697, in get_handle
    check_parent_directory(str(handle))
  File "/databricks/python/lib/python3.9/site-packages/pandas/io/common.py", line 571, in check_parent_directory
    raise OSError(rf"Cannot save file into a no

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

#Initialize SparkSession
spark = SparkSession.builder \
.appName("StructuredStreamingExample") \
.getOrCreate()

#Load data from CSV
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/varshinie.1006@gmail.com/sales_data3.csv")

print("Data Loaded Successfully")

#Transform the data: Add a new column for total amount
df_transformed = df.withColumn("TotalAmount", col("Quantity").cast("int") * col("Price").cast("double"))

print("Data Transformed Successfully")

# Write transformed data to a Delta table
df_transformed.write.format("delta").mode("overwrite").save("/delta/sales_data")

print("Transformed data written to Delta table successfully")

Data Loaded Successfully
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2244438965545018>", line 16, in <module>
    df_transformed = df.withColumn("TotalAmount", (col("Quantity").cast("int") * col("Price").cast("double")))
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/dataframe.py", line 4758, in withColumn
    return DataFrame(self._jdf.withColumn(colName, col._jc), self.sparkSession)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column 